# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import ComplementNB,MultinomialNB,BernoulliNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,accuracy_score,f1_score

nltk.download('punkt') #tokenizer
nltk.download('stopwords') 
nltk.download('wordnet')# lemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/clairegong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/clairegong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/clairegong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM messages', engine)
labels=df.columns[3:]#.tolist()

In [3]:
# load data from database


def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql('SELECT * FROM messages', engine)
    X = df.message.values
    Y = df.iloc[:,3:].values
#     labels=df.iloc[:,3:].columns
    return X,Y#,labels

# df.head()

### 2. Write a tokenization function to process your text data

 #### Three useful methods of  text feature extraction. I don't know when to use each yet.

CountVectorizer - Bag of Words

**TfidfTransformer - TF-IDF values**

TfidfVectorizer - Bag of Words AND TF-IDF values

In [4]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def make_pipe(clf):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', clf)
     ])
    return pipeline

In [6]:
def build_model(clf):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', clf)
     ])
        
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)

    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report(Y_test,Y_preds):
    for i in range(36):
        print(classification_report(Y_test[:,i], Y_preds[:,i]))
#Macro-Average: (class1 + class2+...)/no_class	 #treat classes equally; 
#Micro-Average: total_true/total	#treat instances equally, so classes with larger dataset carry more weight

In [8]:
def multi_f1(Y_test,Y_preds):
    f1_list_weighted=[f1_score(Y_test[:,i], Y_preds[:,i], average='weighted')for i in range(36)]
    print('weighted f1_score',sum(f1_list_weighted)/36)

### 6. Improve your model/GridSearch
Use grid search to find better parameters. 

In [9]:
multi_clf=MultiOutputClassifier(ComplementNB())

def main_cv(clf=multi_clf):
    X, Y = load_data()
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

    model=build_model(clf)

    # train classifier
    model.fit(X_train, Y_train)
    print(model.best_params_)

    # predict on test data
    Y_preds = model.predict(X_test)

    # display averaged f1_score
    multi_f1(Y_test,Y_preds)


# main_cv()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
def main(clf=multi_clf):
    X, Y = load_data()
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

    pipe=make_pipe(clf)

    # train classifier
    pipe.fit(X_train, Y_train)

    # predict on test data
    Y_preds = pipe.predict(X_test)

    # display averaged f1_score
    multi_f1(Y_test,Y_preds)
    return pipe,X_test,Y_test,Y_preds


pipe,X_test,Y_test,Y_preds=main()

weighted f1_score 0.9244240538281583


In [21]:
f1_list=[f1_score(Y_test[:,i], Y_preds[:,i],average='weighted')for i in range(36)]
print('model mean f1_score on test set is: ', sum(f1_list)/36)

model mean f1_score on test set is:  0.9244240538281583


In [11]:
# report(Y_test,Y_preds)

In [12]:
# code for calculating other metrics
# # compute scores for each class
accuracy_list=[accuracy_score(Y_test[:,i], Y_preds[:,i])for i in range(36)]
# f1_list_micro=[f1_score(Y_test[:,i], Y_preds[:,i], average='micro')for i in range(36)]#same value as accuracy
# f1_list_macro=[f1_score(Y_test[:,i], Y_preds[:,i], average='macro')for i in range(36)]

print('average accuracy:',sum(accuracy_list)/36)
# print('micro f1_score',sum(f1_list_micro)/36)

# #macro treat class equally, thus not suitable for imbalanced class
# print('macro f1_score',sum(f1_list_macro)/36)

average accuracy: 0.9339843352659953


In [13]:
score_df=pd.DataFrame({'accuracy':accuracy_list},index=labels)
score_df.head()

,accuracy
related,0.823924
request,0.870156
offer,0.989930
aid_related,0.750534
medical_help,0.909216


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
from sklearn.naive_bayes import ComplementNB,MultinomialNB,BernoulliNB,GaussianNB
#1st model cannot import in Udacity workspace, 2nd trigger warning, third is as good as first
#GaussianNB cannot take sparse matrix
NB_clf_2=MultiOutputClassifier(BernoulliNB(alpha=0.6))
_,_,_,_=main(NB_clf_2)
#alpha=1.0 0.9255590554451469; 0.7, 0.9274249838428733;0.6,0.9276770612976315;
#0.5, 0.9275877839959503; 0.3, 0.9268416314997615

weighted f1_score 0.9276770612976315


### 9. Export your model as a pickle file

In [15]:
from joblib import dump, load
#save
dump(pipe, 'pipe.pkl')
#load
pipe_load = load('pipe.pkl')


In [16]:
# check that pipeline saved sucessfully

Y_preds_load=pipe_load.predict(X_test)
import numpy
numpy.mean(Y_preds_load==Y_preds)

1.0

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.